This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/pose.html).  I recommend having both windows open, side-by-side!

In [ ]:
import numpy as np
import open3d as o3d
from pydrake.all import (AbstractValue, AngleAxis, DiagramBuilder, LeafSystem,
                         MeshcatVisualizer, PiecewisePolynomial, PiecewisePose,
                         PointCloud, RigidTransform, RollPitchYaw, Simulator,
                         StartMeshcat)

from manipulation import running_as_notebook
from manipulation.icp import IterativeClosestPoint
from manipulation.meshcat_utils import AddMeshcatTriad, draw_open3d_point_cloud
from manipulation.mustard_depth_camera_example import MustardPointCloud
from manipulation.open3d_utils import create_open3d_point_cloud
from manipulation.scenarios import (AddIiwaDifferentialIK,
                                    MakeManipulationStation)


In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

# Putting it all together

In the code above, we worked with a point cloud using functions.  To assemble this into a full-stack manipulation system, we need to specify the timing semantics of when those functions are called.  That's precisely what Drake's systems framework provides.  I've introduced two systems below: 
- `MustardIterativeClosestPoint` system that takes the camera inputs and outputs the pose estimate using ICP, and 
- `PickAndPlaceTrajectory` system that takes this pose estimate (and the state of the robot), computes the trajectory, and stores that trajectory in its Context so that it can output the instantaneous command.  

We don't use a `TrajectorySource` here, because the trajectory is not known when we first build the Diagram... the information we need to plan the trajectory requires reading sensors at runtime.

In [ ]:
model_directives = """
directives:
- add_directives:
    file: package://manipulation/clutter_w_cameras.dmd.yaml
- add_model:
    name: mustard
    file: package://drake/manipulation/models/ycb/sdf/006_mustard_bottle.sdf
    default_free_body_pose:
        base_link_mustard:
            translation: [.55, 0.1, 0.09515]
            rotation: !Rpy { deg: [-90, 0, 45] }
"""

# Takes 3 point clouds (in world coordinates) as input, and outputs and estimated pose for the mustard bottle.
class MustardIterativeClosestPoint(LeafSystem):
    def __init__(self):
        LeafSystem.__init__(self)
        model_point_cloud = AbstractValue.Make(PointCloud(0))
        self.DeclareAbstractInputPort("cloud0", model_point_cloud)
        self.DeclareAbstractInputPort("cloud1", model_point_cloud)
        self.DeclareAbstractInputPort("cloud2", model_point_cloud)

        self.DeclareAbstractOutputPort(
            "X_WO", lambda: AbstractValue.Make(RigidTransform()),
            self.EstimatePose)

        self.mustard = MustardPointCloud()
        meshcat.SetObject("icp_scene", self.mustard)

    def EstimatePose(self, context, output):
        # Note: The point cloud processing we do here will be described in more detail in the next chapter.
        pcd = []
        for i in range(3):
            point_cloud = self.get_input_port(i).Eval(context)
            cloud = create_open3d_point_cloud(point_cloud)
            # Crop to region of interest.
            pcd.append(cloud.crop(
                o3d.geometry.AxisAlignedBoundingBox(min_bound=[.4, -.2, 0.001],
                                                    max_bound=[.6, .3, .3])))
        # Merge point clouds.
        merged_pcd = pcd[0] + pcd[1] + pcd[2]
        # Down-sample.
        down_sampled_pcd = merged_pcd.voxel_down_sample(voxel_size=0.005)
        draw_open3d_point_cloud(meshcat, "icp_observations", down_sampled_pcd)

        X_ModelScene, chat = IterativeClosestPoint(
            np.asarray(down_sampled_pcd.points).T,
            self.mustard.xyzs(),
            meshcat=meshcat,
            meshcat_scene_path="icp_scene")

        output.set_value(X_ModelScene.inverse())

class PickAndPlaceTrajectory(LeafSystem):
    def __init__(self, plant):
        LeafSystem.__init__(self)
        self._gripper_body_index = plant.GetBodyByName("body").index()
        self.DeclareAbstractInputPort(
            "body_poses", AbstractValue.Make([RigidTransform()]))
        #            plant.get_body_poses_output_port().Allocate())
        self.DeclareAbstractInputPort("X_WO",
                                      AbstractValue.Make(RigidTransform()))

        self.DeclareInitializationUnrestrictedUpdateEvent(self.Plan)
        self._X_G_traj_index = self.DeclareAbstractState(
            AbstractValue.Make(PiecewisePose()))
        self._wsg_traj_index = self.DeclareAbstractState(
            AbstractValue.Make(PiecewisePolynomial()))

        self.DeclareAbstractOutputPort(
            "X_WG", lambda: AbstractValue.Make(RigidTransform()),
            self.CalcGripperPose)
        self.DeclareVectorOutputPort("wsg_position", 1, self.CalcWsgPosition)

    def Plan(self, context, state):
        X_Ginitial = self.get_input_port(0).Eval(context)[
            int(self._gripper_body_index)]
        X_Oinitial = self.get_input_port(1).Eval(context)
        X_Ogoal = RigidTransform([0, -.6, 0])
        X_GgraspO = RigidTransform(RollPitchYaw(np.pi / 2, 0, 0),
                                   [0, 0.22, 0])
        X_GgraspGpregrasp = RigidTransform([0, -0.08, 0])
        X_OGgrasp = X_GgraspO.inverse()

        X_Gpick = X_Oinitial @ X_OGgrasp
        X_Gprepick = X_Gpick @ X_GgraspGpregrasp
        X_Gplace = X_Ogoal @ X_OGgrasp
        X_Gpreplace = X_Gplace @ X_GgraspGpregrasp

        # I'll interpolate a halfway orientation by converting to axis angle
        # and halving the angle.
        X_GprepickGpreplace = X_Gprepick.inverse() @ X_Gpreplace
        angle_axis = X_GprepickGpreplace.rotation().ToAngleAxis()
        X_GprepickGclearance = RigidTransform(
            AngleAxis(angle=angle_axis.angle()/2.0, axis=angle_axis.axis()),
            X_GprepickGpreplace.translation()/2.0 + np.array([0, -0.3, 0]))
        X_Gclearance = X_Gprepick @ X_GprepickGclearance

        if False:  # Useful for debugging
            AddMeshcatTriad(meshcat, "X_Oinitial", X_PT=X_Oinitial)
            AddMeshcatTriad(meshcat, "X_Gprepick", X_PT=X_Gprepick)
            AddMeshcatTriad(meshcat, "X_Gpick", X_PT=X_Gpick)
            AddMeshcatTriad(meshcat, "X_Gplace", X_PT=X_Gplace)

        # Construct the trajectory.  This is a condensed version of the logic
        # from the previous chapter, adapted to the mustard bottle.
        times = [0]
        poses = [X_Ginitial]
        def add_pose_segment(duration, X_G):
            times.append(times[-1] + duration)
            poses.append(X_G)
        wsg_times = []
        wsg_commands = []
        def wsg_opened():
            wsg_times.append(times[-1])
            wsg_commands.append([0.107])
        def wsg_closed():
            wsg_times.append(times[-1])
            wsg_commands.append([0])

        wsg_opened()
        add_pose_segment(
            10.0 * np.linalg.norm(X_Gprepick.translation() - X_Ginitial.translation()), X_Gprepick)
        add_pose_segment(2, X_Gpick)
        wsg_opened()
        add_pose_segment(2, X_Gpick)  # while the gripper closes
        wsg_closed()
        add_pose_segment(2, X_Gprepick)
        time_to_from_clearance = 10.0*np.linalg.norm(X_GprepickGclearance.translation())
        add_pose_segment(time_to_from_clearance, X_Gclearance)
        add_pose_segment(time_to_from_clearance, X_Gpreplace)
        add_pose_segment(2, X_Gplace)
        wsg_closed()
        add_pose_segment(2, X_Gplace)  # while the gripper opens
        wsg_opened()
        add_pose_segment(2, X_Gpreplace)
        wsg_opened()

        X_G_traj = PiecewisePose.MakeLinear(times, poses)
        state.get_mutable_abstract_state(int(self._X_G_traj_index)).set_value(
            X_G_traj)
        state.get_mutable_abstract_state(int(self._wsg_traj_index)).set_value(
            PiecewisePolynomial.FirstOrderHold(
                wsg_times,
                np.asarray(wsg_commands).reshape(1, -1)))
        print(X_G_traj.end_time())

    def start_time(self, context):
        return context.get_abstract_state(
            int(self._X_G_traj_index)).get_value().start_time()

    def end_time(self, context):
        return context.get_abstract_state(
            int(self._X_G_traj_index)).get_value().end_time()

    def CalcGripperPose(self, context, output):
        # Evaluate the trajectory at the current time, and write it to the
        # output port.
        output.set_value(context.get_abstract_state(int(
            self._X_G_traj_index)).get_value().GetPose(context.get_time()))

    def CalcWsgPosition(self, context, output):
        # Evaluate the trajectory at the current time, and write it to the
        # output port.
        output.SetFromVector(
            context.get_abstract_state(int(
                self._wsg_traj_index)).get_value().value(context.get_time()))


def icp_pick_and_place_demo():
    builder = DiagramBuilder()

    station = builder.AddSystem(
        MakeManipulationStation(model_directives, time_step=0.002))
    plant = station.GetSubsystemByName("plant")

    icp = builder.AddSystem(MustardIterativeClosestPoint())
    builder.Connect(station.GetOutputPort("camera3_point_cloud"),
                    icp.get_input_port(0))
    builder.Connect(station.GetOutputPort("camera4_point_cloud"),
                    icp.get_input_port(1))
    builder.Connect(station.GetOutputPort("camera5_point_cloud"),
                    icp.get_input_port(2))

    plan = builder.AddSystem(PickAndPlaceTrajectory(plant))
    builder.Connect(station.GetOutputPort("body_poses"),
                    plan.GetInputPort("body_poses"))
    builder.Connect(icp.GetOutputPort("X_WO"), plan.GetInputPort("X_WO"))

    robot = station.GetSubsystemByName(
        "iiwa_controller").get_multibody_plant_for_control()

    # Set up differential inverse kinematics.
    diff_ik = AddIiwaDifferentialIK(builder, robot)
    builder.Connect(diff_ik.get_output_port(),
                    station.GetInputPort("iiwa_position"))
    builder.Connect(plan.GetOutputPort("X_WG"),
                    diff_ik.get_input_port(0))

    builder.Connect(plan.GetOutputPort("wsg_position"),
                    station.GetInputPort("wsg_position"))

    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, station.GetOutputPort("query_object"), meshcat)
    diagram = builder.Build()

    simulator = Simulator(diagram)
    context = simulator.get_context()
    plant_context = plant.GetMyContextFromRoot(context)
    diff_ik_context = diff_ik.GetMyMutableContextFromRoot(context)

    q0 = plant.GetPositions(plant_context, plant.GetModelInstanceByName("iiwa"))
    diff_ik.get_mutable_parameters().set_nominal_joint_position(q0)
    diff_ik.SetPositions(diff_ik_context, q0)

    simulator.Initialize()
    if False: # draw the trajectory triads
        X_G_traj = plan.GetMyContextFromRoot(context).get_abstract_state(
            0).get_value()
        for t in np.linspace(X_G_traj.start_time(), X_G_traj.end_time(), 40):
            AddMeshcatTriad(meshcat,
                            f"X_G/({t})",
                            X_PT=X_G_traj.GetPose(t),
                            length=.1,
                            radius=0.004)

    if running_as_notebook:
        meshcat.Flush()
        visualizer.StartRecording()
        simulator.AdvanceTo(plan.end_time(plan.GetMyContextFromRoot(context)))
        visualizer.PublishRecording()
    else:
        simulator.AdvanceTo(0.1)

icp_pick_and_place_demo()
